### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [54]:
TEST_X_FULL_PATH = "hd_X_test.csv"
TEST_Y_FULL_PATH = "hd_y_test.csv"

In [91]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv(TEST_X_FULL_PATH)
y_test = pd.read_csv(TEST_Y_FULL_PATH)

In [92]:
X_test[['Age', 'Sex', 'ChestPainType','Cholesterol','ExerciseAngina',\
       'Oldpeak','ST_Slope']].head(3)

,Age,Sex,ChestPainType,Cholesterol,ExerciseAngina,Oldpeak,ST_Slope
0,63,F,ATA,195,N,0.0,Up
1,53,M,NAP,518,N,0.0,Flat
2,65,M,ASY,0,N,1.2,Flat


In [93]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             230 non-null    int64  
 1   Sex             230 non-null    object 
 2   ChestPainType   230 non-null    object 
 3   Cholesterol     230 non-null    int64  
 4   ExerciseAngina  230 non-null    object 
 5   Oldpeak         230 non-null    float64
 6   ST_Slope        230 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 12.7+ KB


In [95]:
X_test["Age"] = X_test["Age"].astype('int32')
X_test["Cholesterol"] = X_test["Cholesterol"].astype('int32')

In [138]:
import urllib.request
import json
def get_prediction(age, sex, chest_pain_type,cholesterol,exercise_angina,\
                  old_peak,ST_Slope):
    try:
        age = int(age)
        cholesterol = int(cholesterol)
    except e:
        pass
    body = {'Age': age, 'Sex': sex, 'ChestPainType' :chest_pain_type,\
                            'Cholesterol': cholesterol,'Oldpeak': old_peak, \
                            'ExerciseAngina': exercise_angina,\
                             'ST_Slope':ST_Slope} 
    

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [139]:
get_prediction(76, 'F','ATA',500,'N',1.5,'Up')

0.06426117229921083

In [140]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             230 non-null    int32  
 1   Sex             230 non-null    object 
 2   ChestPainType   230 non-null    object 
 3   Cholesterol     230 non-null    int32  
 4   ExerciseAngina  230 non-null    object 
 5   Oldpeak         230 non-null    float64
 6   ST_Slope        230 non-null    object 
dtypes: float64(1), int32(2), object(4)
memory usage: 10.9+ KB


In [141]:
%%time
predictions = X_test[['Age', 'Sex', 'ChestPainType','Cholesterol','ExerciseAngina',\
                  'Oldpeak','ST_Slope']].iloc[:500].apply(lambda x: get_prediction(x[0], x[1], x[2],x[3], x[4], x[5],x[6]))

TypeError: Object of type int32 is not JSON serializable

In [142]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

NameError: name 'predictions' is not defined

Давайте подумаем, где наше api может сломаться

In [119]:
get_prediction('','',None,'',None,None,None)

0.624026673268513